In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from shapely import geometry
from utils import *
tqdm.pandas()

C:\Users\paulo\anaconda3\envs\faces\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
dataset360 = pd.read_pickle('../dataframes/dataset360_detections.pkl')
dataset360.head(2)

,image_path,metadata,fold,phi,theta,r_h,faces,circular_perimeter_faces,detections_mtcnn,detections_viewport,detections_sphere_mtcnn
0,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,-35,78,0.994500,"[[(320, 114), (374, 112), (427, 116), (328, 15...","[[(320, 114), (374, 112), (427, 116), (415, 15...","[[(332, 119), (402, 119), (402, 202), (332, 20...","[[(324, 138), (357, 125), (399, 115), (411, 15...",[]
1,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,../data/dataset_360/FDDB-fold-01\2002_07_19_bi...,FDDB-fold-01,22,-142,0.523074,"[[(1138, 357), (1152, 357), (1166, 356), (1138...","[[(1138, 357), (1152, 357), (1166, 356), (1167...","[[(1140, 362), (1164, 362), (1164, 395), (1140...","[[(1139, 361), (1149, 361), (1160, 360), (1162...",[]


## mAP

In [3]:
def compute_metrics(df, gt_column, pred_column, threshold = 0.5):
    total_faces = df.circular_perimeter_faces.apply(len).sum()

    tps = []
    fps = []

    tp = 0
    fp = 0
    for index , row in df.iterrows(): 
        ground_truths = row['circular_perimeter_faces']
        gt_polys = [geometry.Polygon(adjust_bounds(gt)).convex_hull for gt in ground_truths]

        tests = row[pred_column]   
        for test in tests:               
            ious = []

            for gt_poly in gt_polys:            
                gt_test = geometry.Polygon(adjust_bounds(test)).convex_hull

                iou = gt_poly.intersection(gt_test).area/gt_poly.union(gt_test).area
                ious.append(iou)

            if len(gt_polys) > 0:
                max_iou = np.argmax(ious)

                if ious[max_iou] > threshold:#true positive
                    tp+=1

                    gt_polys.pop(max_iou)

                else:
                    fp += 1
            else: 
                fp += 1

            #print(f'tp: {tp}, fp: {fp}')
            tps.append(tp)
            fps.append(fp)

    tps = np.array(tps)
    fps = np.array(fps)
    
    precision = tps/(tps+fps)
    recall = tps/total_faces

    eval_df = pd.DataFrame({'precision':precision, 'recall':recall})
    
    prec_at_rec = []
    
    ##map
    for recall_level in np.linspace(0.0, 1.0, 11):
        try:
            x = eval_df[eval_df['recall'] >= recall_level]['precision']
            prec = max(x)
        except:
            prec = 0.0
        prec_at_rec.append(prec)
    avg_prec = np.mean(prec_at_rec)

    print('11 point precision is ', prec_at_rec)
    print('mAP is ', avg_prec)

    return avg_prec, precision, recall, prec_at_rec

## Overall

In [4]:
avg_prec, precision, recall, prec_at_rec = compute_metrics(dataset360, 'circular_perimeter_faces', 'detections_viewport') 

11 point precision is  [1.0, 0.6153388822829964, 0.6153388822829964, 0.6148511990754117, 0.6148511990754117, 0.6089432255903534, 0.6089432255903534, 0.6089432255903534, 0.0, 0.0, 0.0]
mAP is  0.4806554399534433


## Close to Poles

In [12]:
poles = dataset360.loc[dataset360.phi.apply(abs) >= 75]
poles.shape

(318, 11)

In [14]:
avg_prec, precision, recall, prec_at_rec = compute_metrics(poles, 'circular_perimeter_faces', 'detections_mtcnn') 

11 point precision is  [0.4444444444444444, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mAP is  0.0404040404040404


## Close to Borders

In [20]:
borders = dataset360.loc[dataset360.theta.apply(abs) >= 160]
borders.shape

(317, 11)

In [22]:
avg_prec, precision, recall, prec_at_rec = compute_metrics(borders, 'circular_perimeter_faces', 'detections_mtcnn') 

11 point precision is  [1.0, 0.7010869565217391, 0.7010869565217391, 0.688212927756654, 0.6732394366197183, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
mAP is  0.3421478434018046
